In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# https://www.kaggle.com/datasets/emreokcular/turkish-song-lyrics
df = pd.read_csv("turkish_song_lyrics.csv") 
df.head()

In [ ]:
df["singer"].unique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isna().sum()

In [ ]:
df.groupby(["singer"])["song"].count().sort_values(ascending=False)

In [ ]:
df.singer.value_counts().plot.bar(title="Sanatçıya Göre Şarkı Sayıları",figsize=(15,8))
plt.xticks(rotation=90)
plt.show()

In [ ]:
sarki_sozleri = df.lyrics.to_numpy()
sarki_sozleri[:2]

In [ ]:
satirlar = sarki_sozleri[0].lower().split("\n")
satirlar[10:20]

In [ ]:
pi = {} # 
A1 = {}

In [ ]:
import string

satir_sonu = "<END>"

for sarki_sozu in sarki_sozleri: # sarki_sozu : tek bir sarkinin tum satirlari
    satirlar = sarki_sozu.lower().split("\n") # satirlar : bir sarkinin satirlari
    
    for satir in satirlar: # bir sarkinin 1 satiri
        # satirdaki kelimeler (kucuk harf, noktalamasiz)
        tokens = satir.translate(str.maketrans("", "", string.punctuation)).split()
        token_sayisi = len(tokens) # satirdaki kelime sayisi
        
        for i in range(token_sayisi):
            token = tokens[i] # token : satirdaki kelime
            if i == 0: # ilk kelime icin;
                # pi dict'inde bu kelime onceden gectiyse frekansini 1 arttir
                # eger gecmediyse frekansini 1 yap
                pi[token] = pi.get(token, 0) + 1
            else:
                onceki_token = tokens[i-1] # bir onceki kelimeyi al
                if onceki_token not in A1:
                    # bir onceki kelime A1'de yoksa ona ait bir list yarat
                    A1[onceki_token] = [] 

                # A1'de oncekli kelimeye ait listeye simdiki kelimeyi ekle
                A1[onceki_token].append(token)
                
                if i == (token_sayisi - 1): # satir sonuna geldiysek
                    # satirin son kelimesi A1'de yoksa A1'de son kelime icin bir list yarat
                    if token not in A1:
                        A1[token] = []
                    # satirin son kelimesinin listesine satir sonu sembolu ekle
                    A1[token].append(satir_sonu)

In [ ]:
pi_sorted = sorted(pi, key=pi.get, reverse=True)

for token in pi_sorted[:10]:
    print(token, ": ", pi[token])

In [ ]:
i = 0
for k in A1:
    if i == 7:
        break
    print(k, ": ", A1[k][:10], "\n")
    i += 1

In [ ]:
pi_total = sum(pi.values())
pi_total

In [ ]:
for token in pi:
    # pi dict'indeki frekanslari normalize ediyoruz
    pi[token] = pi[token] / pi_total

In [ ]:
i = 0
for token in pi:
    if i == 5:
        break
    print(token, pi[token])
    i += 1

In [ ]:
pi_total = sum(pi.values())
pi_total

In [ ]:
# A1'deki key, value ikililerinin formati şu an:

# gelirdi :  ['<END>', '<END>', 'ayrılıksa', 'elimizden', '<END>', '<END>', 'sevişirdik', '<END>', 'her', '<END>', ...] 

# Bunlari dönüştüreceğimiz format:

# gelirdi :  {'<END>': 0.3, 'ayrılıksa': 0.01, 'elimizden': 0.13, ...}

In [ ]:
for token in A1: # token : gelirdi
    tokens_list = A1[token] # tokens_list : ['<END>', '<END>', 'ayrılıksa', ...]
    num_tokens = len(tokens_list)
    
    tokens_dict = {}
    for t in tokens_list: # t : '<END>'
        tokens_dict[t] = tokens_dict.get(t, 0) + 1 # tokens_dict : { '<END>':7, ... }

    for t in tokens_dict: # tokens_dict'teki butun sayilari normalize et
        tokens_dict[t] = tokens_dict[t] / num_tokens
    
    A1[token] = tokens_dict # A1'deki value'ları artık list değil; dictionary


In [ ]:
print(A1["giderdi"], "\n")
print(sum(A1["giderdi"].values()))

In [ ]:
def kelime_sec(d):
    """
    d: {'<END>': 0.7, 'her': 0.1, 'test': 0.2} gibi bir dict'ten 
    dict'teki olasiliklara göre rastgele bir kelime seçecek.
    '<END>'in seçilme olasiligi 0.7, 'test'in secilme olasiligi 0.2,
    'her'in secilme olasiligi 0.1 olacak.
    """
    rastgele_sayi = np.random.random() # rastgele_sayi: [0, 1)
    esik = 0
    for token, prob in d.items():
        esik += prob 
        if esik >= rastgele_sayi:
            return token
    
    raise Exception("kelime_sec fonksiyonu yanlis calisiyor!")

In [ ]:
for _ in range(5):
    token = kelime_sec(pi)
    print(token, pi[token])

In [ ]:
for _ in range(5):
    token = kelime_sec(A1["gelirdi"])
    print(token, A1["gelirdi"][token])

In [ ]:
def soz_yaz(satir_sayisi):
    satirlar = []
    for i in range(satir_sayisi):
        satir = []
        ilk_kelime = kelime_sec(pi)
        satir.append(ilk_kelime)
        
        onceki_kelime = ilk_kelime
        while True:
            yeni_kelime = kelime_sec(A1[onceki_kelime])
            if yeni_kelime == satir_sonu:
                satirlar.append(" ".join(satir))
                break
            satir.append(yeni_kelime)
            onceki_kelime = yeni_kelime
    
    return "\n".join(satirlar)

In [ ]:
print(soz_yaz(4))